In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [8]:
from chromadb import PersistentClient
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

CHROMA_PATH = "./chroma"

embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434",
    model="qwen3"
)

persistent_client = PersistentClient(
    path=CHROMA_PATH,
)

collection_name = "data"
collection = persistent_client.get_or_create_collection(name=collection_name)

vector_store = Chroma(
    client=persistent_client,
    collection_name=collection_name,
    embedding_function=embeddings,
)

# Add documents to the vector store
def add_documents_to_vector_store(documents):
    vector_store.add_documents(
        documents=documents,
        ids=[doc.metadata['source_type'] + "_" + doc.metadata['source'] + "_" + str(doc.metadata['chunk_number']) for doc in documents],
    )
    print(f"Added {len(documents)} documents to the vector store.")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
DATA_PATH = './flat_content'

def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()

    # Add metadata to each document
    for doc in documents:
        doc.metadata['source_type'] = 'markdown'
        doc.metadata['source'] = DATA_PATH

    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)

    # Add metadata to each chunk
    for i, chunk in enumerate(chunks):
        chunk.metadata['chunk_number'] = i + 1

    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    add_documents_to_vector_store(chunks)
    print("Data store generated successfully.")

In [ ]:
if __name__ == "__main__":
    main()

Split 1479 documents into 13516 chunks.
|3.4.1 | - | 3.1.2 | |tensorflow |2.13.0 | - | 2.7.0, 2.10.0 | |texlive |2023 | - | 2020 | {{< /table >}}
{'source': './flat_content', 'source_type': 'markdown', 'start_index': 2602, 'chunk_number': 10}
